In [9]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals
import warnings
warnings.filterwarnings('ignore')


import re
import os, sys

import numpy as np # linear algebra
import spacy
from scipy.special import softmax

import shutil

import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer

from config import SEED, PARTIAL_TRAIN, TEST_SIZE, NUM_LABELS 
from config import MAX_SEQUENCE_LENGTH, NUM_EPOCH, LEARNING_RATE, BATCH_SIZE
from config import ACCUMULATION_STEPS, INPUT_DIR, WORK_DIR, TOXICITY_COLUMN, DATA_DIR
from config import BERT_MODEL_NAME, FINE_TUNED_MODEL_PATH, BERT_MODEL_PATH

from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam
from utils import set_seed, convert_lines_onfly, preprocess

device = torch.device('cpu')

In [2]:
convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt',
BERT_MODEL_PATH + 'bert_config.json',
WORK_DIR + 'pytorch_model.bin')

shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

Building PyTorch model from configuration: {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Converting TensorFlow checkpoint from /Users/matthewvu/Desktop/ToxicDetection/input/uncased_L-12_H-768_A-12/bert_model.ckpt
Loading TF weight bert/embeddings/LayerNorm/beta with shape [768]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [768]
Loading TF weight bert/embeddings/position_embeddings with shape [512, 768]
Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 768]
Loading TF weight bert/embeddings/word_embeddings with shape [30522, 768]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [768]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/gamma 

'../working/bert_config.json'

In [3]:
from pytorch_pretrained_bert import BertConfig

bert_config = BertConfig('../input/uncased_L-12_H-768_A-12/'+'bert_config.json')

In [4]:
## instantiate bert pretrained model and tokenizer
model = BertForSequenceClassification(bert_config, num_labels=NUM_LABELS)
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None, do_lower_case=True)

## load saved model
model.load_state_dict(torch.load(FINE_TUNED_MODEL_PATH,map_location=torch.device('cpu'))) 
for p in model.parameters():
    p.requires_grad = False

## use gpu
##model.cuda()

def predict_toxicity(sentence: str) -> float :
    """
    predict the toxicity level from a sentence
    """    

    model.eval()
    
    X = np.array([str(sentence)])
    test_preds = torch.zeros((len(X)))
    
    Xp = convert_lines_onfly(X, MAX_SEQUENCE_LENGTH, tokenizer)
    y_pred = model(torch.from_numpy(Xp).to(device))
    test_preds[0] += torch.sigmoid(y_pred[0,0].cpu())

    return float(test_preds[0])

In [5]:
# Preprocess text (username and link placeholders)
def preprocess_sentiment(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task = 'sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
sentiment_tokenizer = AutoTokenizer.from_pretrained(MODEL)

# PT
sentiment_model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# label mapping
NEG = 'negative'
NET = 'neutral'
POS = 'positive'
labels = [NEG, NET, POS]

label2id = {k:v for k, v in zip(labels, range(3))}
id2label = {k:v for k, v in zip(range(3), labels)}

def predict_sentiment(sentence: str) :
    text = preprocess_sentiment(sentence)
    encoded_input = sentiment_tokenizer(text, return_tensors='pt')
    output = sentiment_model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    sentiment = {id2label[idx]:s for idx, s in enumerate(scores)}

    return sentiment

In [6]:
import en_core_web_sm
nlp = en_core_web_sm.load()

def split_text(sentence): 
    doc = nlp(sentence)
    sentences = [sent.text.strip() for sent in doc.sents]
    return sentences

In [33]:
# sentence = "Oh sh*t!! What an awesome goal, I nearly missed it…"
#sentence = "Yet call out all Muslims for the acts of a few will get you pilloried.   So why is it okay to smear an entire religion over these few idiots?  Or is this because it's okay to bash Christian sects?"
#sentence = "Sorry to have to do this, but just to see if profanity filtering is enabled"
sentence = "haha you guys are a bunch of losers."

In [35]:
print("Sentence: ", sentence)
print("Toxicity: ", predict_toxicity(sentence))
print("Sentiment: ", predict_sentiment(sentence))
print("Splitted Sentence: ", split_text(sentence))

Sentence:  haha you guys are a bunch of losers.
Toxicity:  0.9349055290222168
Sentiment:  {'negative': 0.97574496, 'neutral': 0.020578168, 'positive': 0.0036769167}
Splitted Sentence:  ['haha you guys are a bunch of losers.']


In [11]:
def handle_edge_case(sentence: str, toxic_threshold=0.3) -> float :
    sentences = split_text(sentence)
    for i in range(len(sentences)) :
        truncated = " ".join(sentences[:i] + sentences[i+1:])
        toxicity = predict_toxicity(truncated)
        sentiment = predict_sentiment(truncated)
        if toxicity < toxic_threshold and sentiment[POS] > sentiment[NEG] :
            return toxicity
    return predict_toxicity(sentence)

def predict_with_combined_toxicity_sentiment(sentence: str) -> float :
    toxicity = predict_toxicity(sentence)
    sentiment = predict_sentiment(sentence)
    
    score = toxicity
    
    if toxicity > 0.9 :
        if sentiment[NEG] > sentiment[POS] :
            score = toxicity
        else :
            score = handle_edge_case(sentence, toxic_threshold=0.1)
    elif toxicity > 0.3 :
        if sentiment[NEG] > sentiment[POS] :
            score = toxicity
        else :
            score = handle_edge_case(sentence, toxic_threshold=0.5)
    else : # toxicity < 0.3
        score = toxicity
            
    return score

In [36]:
print(sentence)
print(predict_with_combined_toxicity_sentiment(sentence))

haha you guys are a bunch of losers.
0.9349055290222168
